<a href="https://colab.research.google.com/github/LuchoValr/Machine_Learning/blob/main/Proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importacion de librerias

In [1]:
import pandas as pd

##Carga de datos

In [84]:
data=pd.read_csv('https://raw.githubusercontent.com/labeconometria/MLxE/main/proyectos1er/dataset_1.csv')
data.head()

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02


##Exploración inicial

In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1174 non-null   int64  
 1   gender     1174 non-null   object 
 2   wage       930 non-null    float64
 3   purchased  1174 non-null   int64  
 4   birth_day  1174 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 46.0+ KB


In [86]:
data.describe()

,user_id,wage,purchased
count,1174.000000,930.000000,1174.000000
mean,499.988927,71948.387097,0.404600
std,287.648865,34112.616989,0.491024
min,1.000000,15000.000000,0.000000
25%,252.250000,45500.000000,0.000000
50%,499.500000,71250.000000,0.000000
75%,749.750000,89500.000000,1.000000
max,1000.000000,152500.000000,1.000000


##Revisión de duplicados

In [87]:
data['user_id'].duplicated().sum()

174

In [88]:
data = data.drop_duplicates().reset_index(drop=True)
data

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02
...,...,...,...,...,...
995,305,Female,22500.0,1,1969-03-09
996,636,Male,76500.0,0,1970-08-26
997,320,Female,143500.0,1,1970-04-07
998,721,Male,148000.0,1,1993-11-23


##Transformación de tipos de datos

In [89]:
from numpy import int32
data=data.astype({'user_id':'int32','purchased':'int32','wage':'float32'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1000 non-null   int32  
 1   gender     1000 non-null   object 
 2   wage       786 non-null    float32
 3   purchased  1000 non-null   int32  
 4   birth_day  1000 non-null   object 
dtypes: float32(1), int32(2), object(2)
memory usage: 27.5+ KB


##Enriquecimiento de datos

In [101]:
age = data['birth_day'].str.split('-', expand=True)
age = age[0].astype('int32')
age

0      1983
1      1972
2      1981
3      1983
4      1967
       ... 
995    1969
996    1970
997    1970
998    1993
999    1984
Name: 0, Length: 1000, dtype: int32

In [102]:
data['age'] = 2023 - age
data.head()


,user_id,gender,wage,purchased,birth_day,age
0,217,Female,NaN,0,1983-08-06,40
1,226,Female,146500.0,1,1972-01-28,51
2,697,Male,NaN,1,1981-12-12,42
3,743,Female,NaN,0,1983-05-26,40
4,63,Female,140500.0,1,1967-11-02,56


##Missing values

In [104]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1000 non-null   int32  
 1   gender     1000 non-null   object 
 2   wage       786 non-null    float32
 3   purchased  1000 non-null   int32  
 4   birth_day  1000 non-null   object 
 5   age        1000 non-null   int32  
dtypes: float32(1), int32(3), object(2)
memory usage: 31.4+ KB


In [114]:
data['wage'].median()

71000.0

In [107]:
data.pivot_table(index=['gender','age'],values='wage',aggfunc=['mean','median'])

mean    median
                     wage      wage
gender age                         
Female 20    55875.000000   57500.0
       21    47400.000000   27500.0
       22    65500.000000   68000.0
       23    42000.000000   41000.0
       24    52833.332031   56250.0
...                   ...       ...
woman  58   108500.000000  108500.0
       60    70833.335938   76000.0
       61    43500.000000   43500.0
       62   103500.000000  103500.0
       63    31500.000000   31500.0

[150 rows x 2 columns]

In [112]:
data['wage'] = data['wage'].fillna(data.groupby(['gender','age'])['wage'].transform('median'))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1000 non-null   int32  
 1   gender     1000 non-null   object 
 2   wage       989 non-null    float32
 3   purchased  1000 non-null   int32  
 4   birth_day  1000 non-null   object 
 5   age        1000 non-null   int32  
dtypes: float32(1), int32(3), object(2)
memory usage: 31.4+ KB


In [116]:
data['wage'] = data['wage'].fillna(71000.0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1000 non-null   int32  
 1   gender     1000 non-null   object 
 2   wage       1000 non-null   float32
 3   purchased  1000 non-null   int32  
 4   birth_day  1000 non-null   object 
 5   age        1000 non-null   int32  
dtypes: float32(1), int32(3), object(2)
memory usage: 31.4+ KB


##Outliers

In [117]:
q1 = data["wage"].quantile(0.25)
q3 = data["wage"].quantile(0.75)
IQR = q3 - q1
lim_sup = q3 + (1.5 * IQR)
lim_inf = q1 - (1.5 * IQR)

In [118]:
lim_inf,lim_sup

(-9250.0, 148750.0)

##Datos inconsistentes

##Manejo de memoria

##Clasificación de datos

In [135]:
data['cat_age'] = pd.cut(data.age, bins=[0, 25, 30, 60, float('Inf')], labels=['teen', 'young','adult', 'old'])
data.head()

,user_id,gender,wage,purchased,birth_day,age,cat_age
0,217,Female,66000.0,0,1983-08-06,40,adult
1,226,Female,146500.0,1,1972-01-28,51,adult
2,697,Male,108500.0,1,1981-12-12,42,adult
3,743,Female,66000.0,0,1983-05-26,40,adult
4,63,Female,140500.0,1,1967-11-02,56,adult
